In [3]:
data = None

with open('fas_crse_cat.txt', 'r') as f:
    raw_content = f.read()
    data = f.read().splitlines()

In [4]:
from bs4 import BeautifulSoup

In [5]:
raw_content

with open('fas_crse_cat.html', 'r') as f:
    raw_content = f.read()
    
soup = BeautifulSoup(raw_content, 'html.parser')

In [6]:
# all content will be in div.t
# so extract this
elts = soup.find_all('div', class_='t')

""

''

In [7]:
# the .fs2 will be the title
# following .fs3 will be descriptive strings

groups = []
current_group = None

for elt in elts:    
    if 'fs2' in elt['class']:
        # .fs2 marks course titles
        
        # first, ensure there's something actually here
        # FIXME could be bugs here from page breaks
        if elt.string is None or elt.string.strip() == "":
            continue
            
        # ignore table of contents
        
        # start a new group
        current_group = dict()
        groups.append(current_group)
        
        current_group['name'] = elt.string
        current_group['strings'] = []
    elif elt is not None:
        # these are probably .fs{3,4,5,...}
        # these are descriptive strings
        
        # error checking
        if current_group is None:
            continue
        if elt.string is None:
            continue
            
        # remove page footers
        BANNED_STRINGS = (
            'HARVARD UNIVERSITY',
            '9/4/2017 0:39 AM',
            '9/24/2017 0:39 AM',
            'HARVARD UNIVERSITY 9/24/2017 0:39 AM'
        )
        if elt.string in BANNED_STRINGS:
            continue
            
        current_group['strings'].append(elt.string)

In [12]:
import re 

for group in groups[0:10]:
    # try to extract a course schedule
    times = []
    for string in group['strings']:
        matcher = re.compile("[MTWRF]{1,5} \d{4} [AP]M - \d{4} [AP]M")
        times.append(matcher.findall(string))
        
    flattened_times = sum(times,[])
    
    group['times'] = flattened_times
    
    
    # try to extract a semester
    for string in group['strings']:
        semester = re.search("20\d\d ((Fall)|(Spring))", string)
        if semester is not None:
            # just find the first one then quit
            group['semester'] = semester.group()
            break
        

In [13]:
groups[0:10]

[{'name': u'Table of Contents',
  'strings': [u'African and African American Studies'],
  'times': []},
 {'name': u'Subject: Hausa', 'strings': [], 'times': []},
 {'name': u'Hausa     AA ',
  'semester': u'2017 Fall',
  'strings': [u'John Mugane  ',
   u'2017 Fall (4 Credits)',
   u'Schedule: ',
   u'TBD',
   u'Instructor Permissions:',
   u'None',
   u'Enrollment Cap:',
   u'n/a',
   u'A study of Hausa a most widely used native language and lingua franca in West Africa at the Elementary level (First',
   u'year part 1). Contact hours supplemented by digital resources. Emphasis on written expression, reading ',
   u'comprehension, and oral fluency. Students must complete both terms of this course (parts AA and AB) within the ',
   u'same academic year in order to receive full Elementary Hausa course credit.',
   u'Course Notes:',
   u'Languages in the tutorial program are offered when there is demonstrated ',
   u'curricular or academic need on the part of the student, and when suitabl

In [14]:
# dump data to file

import json
with open('courses_rough.json', 'w') as outfile:
    json.dump(groups, outfile)

In [15]:
# TODO: extract more info